# Rasters (multiple)
###### bcgov-rs-workshop-2023

In this notebook, we will look at ways to load, analyze, and display rasters using Python.

## Setup

We'll start by installing a few additional modules to our environment that we'll use later. Custom environments do not persist between Google Colab sessions, so you will need to install these dependencies each time you start/restart the runtime.

In [4]:
!pip install planetary-computer pystac-client stackstac[viz] xarray-spatial

  Using cached planetary_computer-0.4.9-py3-none-any.whl (17 kB)
  Using cached pystac_client-0.6.0-py3-none-any.whl (30 kB)
  Using cached stackstac-0.4.3-py3-none-any.whl (62 kB)
  Using cached xarray_spatial-0.3.5-py3-none-any.whl (10.9 MB)
  Using cached pystac-1.6.1-py3-none-any.whl (146 kB)
  Using cached pydantic-1.10.5-cp39-cp39-win_amd64.whl (2.2 MB)
  Using cached rasterio-1.3.6-cp39-cp39-win_amd64.whl (22.3 MB)
  Using cached xarray-2023.2.0-py3-none-any.whl (975 kB)
  Using cached cachetools-4.2.4-py3-none-any.whl (10 kB)
  Using cached mercantile-1.2.1-py3-none-any.whl (14 kB)
  Using cached ipyleaflet-0.17.2-py3-none-any.whl (3.7 MB)
  Using cached aiohttp-3.8.4-cp39-cp39-win_amd64.whl (323 kB)
  Using cached datashader-0.14.4-py2.py3-none-any.whl (18.2 MB)
  Using cached yarl-1.8.2-cp39-cp39-win_amd64.whl (56 kB)
  Using cached multidict-6.0.4-cp39-cp39-win_amd64.whl (28 kB)
  Using cached frozenlist-1.3.3-cp39-cp39-win_amd64.whl (34 kB)
  Using cached async_timeout-4.0.

In [3]:
!pip install matplotlib==3.1.1

     ---------------------------------------- 37.8/37.8 MB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for matplotlib
Failed to build matplotlib
  Attempting uninstall: matplotlib
    Found existing installation: matplotlib 3.5.2
    Uninstalling matplotlib-3.5.2:
      Successfully uninstalled matplotlib-3.5.2
  Running setup.py install for matplotlib: started
  Running setup.py install for matplotlib: finished with status 'error'
  Rolling back uninstall of matplotlib
  Moving to c:\users\matt\anaconda3\lib\site-packages\__pycache__\pylab.cpython-39.pyc
   from C:\Users\Matt\AppData\Local\Temp\pip-uninstall-4_eyu_kc\pylab.cpython-39.pyc
  Moving to c:\users\matt\anaconda3\lib\site-packages\matplotlib-3.5.2-py3.9-nspkg.pth
   from C:\Users\Matt\AppData\Local\Temp\pip-uninstall-tqgmdcjf\matplotlib-3.5.2-py3.9-nspkg.pth
  Moving to c:\users\matt\anaconda3\lib\site-packages\matplotlib-

  error: subprocess-exited-with-error
  
  python setup.py bdist_wheel did not run successfully.
  exit code: 1
  
  [501 lines of output]
  Edit setup.cfg to change the build options
  
  BUILDING MATPLOTLIB
    matplotlib: yes [3.1.1]
        python: yes [3.9.13 | packaged by conda-forge | (main, May 27 2022,
                    16:50:36) [MSC v.1929 64 bit (AMD64)]]
      platform: yes [win32]
  
  OPTIONAL SUBPACKAGES
   sample_data: yes [installing]
         tests: no  [skipping due to configuration]
  
  OPTIONAL BACKEND EXTENSIONS
           agg: yes [installing]
         tkagg: yes [installing; run-time loading from Python Tcl/Tk]
        macosx: no  [Mac OS-X only]
  
  OPTIONAL PACKAGE DATA
          dlls: no  [skipping due to configuration]
  
  C:\Users\Matt\anaconda3\lib\site-packages\setuptools\dist.py:287: SetuptoolsDeprecationWarning: The namespace_packages parameter is deprecated, consider using implicit namespaces instead (PEP 420).
    warnings.warn(msg, SetuptoolsDe

Import all the necessary dependencies.

In [5]:
import folium
import json
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import planetary_computer as pc
import pystac_client
import rasterio
from rasterio.windows import from_bounds
from rasterio.warp import transform_bounds
import requests
import stackstac
import xarray as xr
import xrspatial.multispectral as ms

## Xarray

We have looked at a few common Python data structures, like:
- [numpy](https://numpy.org/)
- [pandas](https://pandas.pydata.org/)

We'll look at one more: [xarray](https://docs.xarray.dev/en/stable/). Xarrays are *labeled* arrays, allowing us to apply named dimensions, coordinates, and attributes to numpy arrays.

You can convert several data structures to xarray. Here, we will set up our demo data using numpy arrays. `arr1` contains integers 0 - 99 in a [10, 10] shape, and `arr2` contains all value 255 in a [10, 10] shape. We then stack the arrays into a single array of shape [10, 10, 2]. You can think of this like a 2-band raster. We will actually consider that arr1 and arr2 represent images collected at two different times in the rest of this demo.

In [ ]:
arr1 = np.arange(100).reshape([10, 10]).astype("uint8")
arr2 = (np.ones([10, 10]) * 255).astype("uint8")
stacked_arr = np.dstack([arr1, arr2])

print(f"arr1: {arr1}")
print(f"arr2: {arr2}")
print(f"stacked_arr shape: {stacked_arr.shape}")

The stacked array above has no associated metadata, it is simply two arrays that happen to be stacked together.

When we convert the numpy array to xarray, we can add named `dimensions` to the unnamed numpy dimensions. In this case, there are three dimensions implied by the shape `[10, 10, 2]`. For our purposes, we will name those dimensions: `["lat", "lon", "time"]`.

There will be 10 latitude values, 10 longitude values, and 2 time values. The actual values we will use for these dimension values are called `coordinates`. The values do not necessarily have to be equally spaced, but for this demo we can create equally spaced lat/lon arrays using [np.linspace()](https://numpy.org/doc/stable/reference/generated/numpy.linspace.html). We can also convert a simple list of date-like values to an array of date datatype using [pd.to_datetime()](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html).

The last step is to create the actual xarray DataArray using [xr.DataArray()](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.html). Notice all the extra built-in functionality when we print the xarray object representation.

In [ ]:
min_y = 53
max_y = 55
min_x = -124
max_x = -122

lat_values = np.linspace(min_y, max_y, 10)
lon_values = np.linspace(min_x, max_x, 10)
time_values = pd.to_datetime(["2000-01-01", "2020-01-01"])

xarr = xr.DataArray(
    stacked_arr,
    dims=("lat", "lon", "time"),
    coords={
        "lat": lat_values,
        "lon": lon_values,
        "time": time_values
    }
)
xarr

The xarray contains the original data, plus the extra metadata we've specified. You can see and manipulate the data as you would a numpy array using the `xarr.data` property.

In [ ]:
xarr.data

And we see the dimensions and coordinates through `xarr.dims` and `xarr.coords`.

In [ ]:
print('Dimensions:', xarr.dims)
xarr.coords

xarray allows us to select data based on the dimensions using the [xarr.sel()](https://docs.xarray.dev/en/stable/generated/xarray.DataArray.sel.html) method. Below, we use numpy's [squeeze()](https://numpy.org/doc/stable/reference/generated/numpy.squeeze.html) method to remove an unnecessary one-length axis (i.e. the resulting shape of `xarr.sel(time="2000")` is `[10, 10, 1]` and printing it is ugly. You can try it if you want. Using squeeze, the shape is changed to `[10, 10]` and it prints nicely.). Notice that we have a new xarray and the coordinates reflect only the selected data.

In [ ]:
xarr.sel(time="2000").squeeze()

We can combine multiple selectors. Also, we can select values within a range using a [slice](https://docs.python.org/3/library/functions.html#slice) object. Here, we select data from 2000 and less than 54 degrees latitude.

In [ ]:
xarr.sel(time="2000", lat=slice(0, 54)).squeeze()

Most xarrays are plottable:

In [ ]:
xarr.sel(time="2000").plot(cmap="gray")

In [ ]:
xarr.sel(time="2020").plot(cmap="gray")

One very handy feature of xarrays is the ability to interpolate along any axis. Below, we interpolate an array for 2001 from the provided values of 2000 and 2020. Try changing the interpolation year and observe the changes in the resulting array.

In [ ]:
xarr.interp(time="2009").plot()

folium doesn't understand xarray, but it does support numpy. So, in order to display on a folium map, we need to do some manipulation beforehand. Notice how we can reference coordinates (e.g. `xarr.lat`) and turn min/max values into floating point numbers. Below, we select the data corresponding to 2000, but we could choose 2020, or interpolate new values, as well.

In [ ]:
m = folium.Map(location=[54, -123], zoom_start=5)

bounds = [
    [
        float(xarr.lat.min().data),
        float(xarr.lon.min().data)
    ],
    [
        float(xarr.lat.max().data),
        float(xarr.lon.max().data)
    ]
]

img_arr = xarr.sel(time="2000").data.squeeze()

folium.raster_layers.ImageOverlay(img_arr, bounds).add_to(m)

m

## STAC Metadata

The [SpatioTemporal Asset Catalog](https://stacspec.org/en) (STAC) specification is an emerging standard for describing spatiotemporal geospatial assets (e.g. most remote sensing data). In addition to an [API specification](https://github.com/radiantearth/stac-api-spec/) that we can mostly ignore, the STAC spec introduces a few data structures:

- [STAC Items](https://github.com/radiantearth/stac-spec/blob/master/item-spec/item-spec.md): GeoJSON features including some well-defined properties. For example, each STAC item must include:
  - `id`
  - `geometry`
  - `bbox`
  - `properties` section (which must include `datetime`)
  - `assets` section
  - plus some others. You can see a minimal example [here](https://github.com/radiantearth/stac-spec/blob/master/examples/simple-item.json).
- [STAC Catalogs](https://github.com/radiantearth/stac-spec/tree/master/catalog-spec): a logical grouping containing other catalogs, collections, or items
- [STAC Collections](https://github.com/radiantearth/stac-spec/tree/master/collection-spec): similar to a catalog, but includes additional information about what metadata each contained item should share

We will mostly focus on **STAC Items** contained within **STAC Collections**.

We will use the Microsoft Planetary Computer STAC API. The API documentation is [here](https://planetarycomputer.microsoft.com/docs/reference/stac/). Let's start by listing the collections. The entire list is quite long, so we'll just print the first collection. Notice that there is quite a lot of metadata for each collection!

In [ ]:
root_url = "https://planetarycomputer.microsoft.com/api/stac/v1/"
request_url = f"{root_url}/collections"

response = requests.get(request_url)

response_json = response.json()

first_collection = response_json["collections"][0]

print(json.dumps(first_collection, indent=2))

We can list the collection names, to see if there is something of interest:

In [ ]:
for collection in response_json["collections"]:
  print(collection["id"])

Let's focus on `sentinel-2-l2a`. When we know what we are looking for, we can use the `/collections/sentinel-2-l2a` endpoint to retrieve only the collection information of interest.

In [ ]:
request_url = f"{root_url}/collections/sentinel-2-l2a"

response = requests.get(request_url)

response_json = response.json()

print(json.dumps(response_json, indent=2))

We can request STAC Items from the collections using the `/collections/sentinel-2-l2a/items` endpoint. Notice that we only retrieve the first 10 items. If you were so inclined, there is a `next` link near the end of the response. You could follow it for the next 10 items, and so on. This is called `pagination`.

In [ ]:
request_url = f"{root_url}/collections/sentinel-2-l2a/items"

response = requests.get(request_url)

response_json = response.json()

print(json.dumps(response_json, indent=2))

You might imagine it's not efficient to find items of interest by paging through results. Luckily, there is a `/search` endpoint we can use to narrow down the results.

Notice that there are both `POST` and `GET` endpoints for `/search` in the Planetary Computer API. REST API purists will tell you that GET requests are for retrieving data, and POST requests are for creating new data. However, POST requests may also be used for retrieving data, and are able to circumvent one of the limitations associated with GET requests, namely the URL length limits imposed by various clients.

Remember that when we send a GET request, the parameters are encoded in the URL. For example, if out parameters are `id` = `a`, and `name` = `b`, then the sent request URL will end with `?id=a&name=b`. You can imagine that with complex parameters, especially those including geometries, the URL may become extremely long.

`POST` requests allow us to send a JSON body (like a Python dictionary) packaged alongside the request, outside of the URL. We will focus on the `POST` search endpoint in this workshop, just be aware that you can use `GET` for simple queries.

Let's start by setting up some query parameters. You can see examples of available query parameters in the API documentation, or in the [STAC API spec](https://github.com/radiantearth/stac-api-spec/tree/main/item-search#query-parameter-table). Here we will set up a bounding box and datetime query.

In [ ]:
collection = "sentinel-2-l2a"
dt = "2022-05-01T00:00:00Z/2022-08-01T00:00:00Z"
bbox = [
  -122.8222,
  53.8864,
  -122.8070,
  53.8984
]

data = {
    "collections": [collection],
    "datetime": dt,
    "bbox": bbox
}

request_url = f"{root_url}/search"

response = requests.post(request_url, data=json.dumps(data))
print(f"Status code: {response.status_code}")
response_json = response.json()
print(f"Feature count: {len(response_json['features'])}")
print(response_json)

If our query was successful (i.e. status code = 200), we should have a response containing STAC items within the `features` key. Let's inspect the first STAC item. Notice that it is a Python dictionary with, among others, `assets` and `properties` keys.

Properties contains numerous pieces of descriptive metadata, which may vary from collection to collection, but should be somewhat consistent within a collection.

Assets contain links pointing to actual data. In this collection, there are assets for individual bands (e.g. `B01`, `B02`, etc.) as well as derived data (e.g. `rendered_preview`). We will display `rendered_preview` assets later.

In [ ]:
response_json["features"][0]

We can print the URLs (a.k.a. "hrefs") from each `rendered_preview` asset like so. You may notice that rather than referring to the location of a png file, the links actually point to a different api: [Data API](https://planetarycomputer.microsoft.com/docs/reference/data/). Regardless, you can click on any of the links to see the `rendered_preview` asset in your browser. 

In [ ]:
for feature in response_json["features"]:
  print(feature["assets"]["rendered_preview"]["href"])

We can also show individual assets like this:

In [ ]:
from IPython.display import Image
Image(url=response_json["features"][0]["assets"]["rendered_preview"]["href"], width=300)

The image above is a low resolution preview asset. The raw data is much larger, with each band on the scale of a few hundred MB. We could download each individual band asset (e.g. "B04", "B03", "B02", etc.) and combine into a huge RGB image. In this case, the data provider (Microsoft) has provided a `visual` asset, containing a processed 3-band RGB composite, so we can use that.

Rather than downloading the entire image, however, we will download only the pixels within an area of interest to keep data transfer manageable. Traditional images do not support such partial reads. This is where [Cloud-Optimized GeoTIFFs](https://www.cogeo.org/) (COGs) come in. COGs support "range requests", meaning that we can request only certain parts of a cleverly organized file. COGs otherwise behave the same as traditional GeoTIFFs. You can spot COGs within STAC Items because the asset will be denoted: `'type': 'image/tiff; application=geotiff; profile=cloud-optimized'`

Rasterio supports windowed COG reads. Below, we `open` the url as usual (notice that we can open the file from a url rather than a file on disk), project a bounding box of our area of interest (using `transform_bounds()`), create a `window` object from the bounds (using `from_bounds()`), and read the pixels of interest by making use of the `window` argument (`src.read(i + 1, window=window)`). Notice that this operation completes much faster than downloading a several hundred MB image and clipping out the pixels of interest!

Finally, notice that we `sign` the asset href. The Planetary Computer uses Shared Access Signature (SAS) tokens to grant access to data directly from Azure Blob storage. For our uses, we can simply `sign` URLs without authenticating our account first. If you are going to work with the Planetary Computer on a larger scale, you may run into rate limiting unless you authenticate first and sign your URLs using a personal token. [More information](https://planetarycomputer.microsoft.com/docs/concepts/sas/).

In [ ]:
stac_item = response_json["features"][1]
signed_url = pc.sign(stac_item["assets"]["visual"]["href"])

with rasterio.open(signed_url) as src:
  print(src.profile)
  utm_bounds = transform_bounds("EPSG:4326", src.crs, *bbox)
  window=from_bounds(*utm_bounds, src.transform)

  arrs = []
  for i in range(3):
    arr = src.read(i + 1, window=window)
    arrs.append(arr)

  stacked_arr = np.dstack(arrs)

stacked_arr.shape

In [ ]:
plt.imshow(stacked_arr)

## stackstac

[stackstac](https://stackstac.readthedocs.io/en/latest/) provides a convenient way to interact with STAC assets, and uses several pieces of technology that we've covered today.

First, let's query the Planetary Computer again. This time, we'll use `pystac_client`, which is a wrapper for STAC APIs. You could similarly query the API with `requests` as we have done previously. Notice that `pystac_client` outputs are rendered nicely in notebooks.

In [ ]:
catalog = pystac_client.Client.open('https://planetarycomputer.microsoft.com/api/stac/v1')
catalog

We can retrieve the collection metadata:

In [ ]:
collection = catalog.get_collection("sentinel-2-l2a")
collection

STAC Collections have an `extent` property with temporal and spatial bounds. Let's inspect the temporal extent. Values of `None` indicate an open-ended extent. In this case, there is a start date (when the satellite/constellation became operational) and an open-ended end date.

In [ ]:
collection.extent.temporal.to_dict()

`pystac_client` provides a `search` method, which is analogous to the STAC API `/search` endpoint we have used previously. Here, we will use an `intersects` query (matching the point geometry), falling in the collection's temporal extent, and containing less than 20% cloud cover (as recorded in the STAC Item's `eo:cloud_cover` property).

The given point is the location of the Site C dam, and we will eventually create a time lapse animation for it.

In [ ]:
latlng_point = [-120.9143, 56.1947]

collection = collection
dt = "2015-06-27T10:25:31Z/.."
intersects = {
  "coordinates": latlng_point,
  "type": "Point"
}

search = catalog.search(
    collections=[collection],
    datetime=dt,
    intersects=intersects,
    query={
        "eo:cloud_cover": {
            "lt": "20"
        }
    }
)

We can sign the search results in the same way as a string URL:

In [ ]:
items = pc.sign(search)
len(items)

And we can inspect those items returned by the search. Notice that the `href` values in assets have a long token appended to the URL):

In [ ]:
items[0]

We will want an area of interest around our point, but first we need to project the lat/long coordinates. And, we may as well use the same projection that the STAC Item assets use.

In [ ]:
from pyproj import Transformer

epsg = items[0].properties["proj:epsg"]
print(f"Item EPSG: {epsg}")

transformer = Transformer.from_crs(4326, epsg, always_xy=True)

espg_point = transformer.transform(*latlng_point)
espg_point

Finally, we will get to stackstac.

stackstac will create an organized xarray dataset from a list of STAC Items. Here, we create the `stack` from our items, clipping the dataset to the bounds coordinates (the point +/- a radius value), and specifying a resolution value. We use 10, which corresponds to the 10 metre resolution of the Sentinel-2 RGB bands.

Please do not use a larger radius value in this workshop. Eventually, we will download the images and we want to be conscious of bandwidth.

In [ ]:
radius = 1000 # do not use a larger value
bounds = [
    espg_point[0] - radius,
    espg_point[1] - radius,
    espg_point[0] + radius,
    espg_point[1] + radius,
]
stack = stackstac.stack(items, bounds=bounds, resolution=10)
stack

Some points to notice above:
- the stack was created quickly. No data has been processed or stored at this point. The xarray data structure has been created `lazily`, meaning it only does things when it has to. It has enough information to know its shape, coordinates, attributes, etc. In order to populate the xarray with actual data in memory, we will call the `.compute()` method. 
- the shape of the dataset is (182, 17, 201, 201), which translates to 182 images 201 x 201 pixels, and 17 bands deep. The exact shape you see may be different.
- there are 46 coordinates, but the really important ones are: time, band, x, y
- if downloaded in its entirety, this dataset would be quite large because there is a lot of unnecessary data (i.e. we will not use all 17 bands).

Let's start by selecting only the bands of interest:

In [ ]:
rgb = stack.sel(band=["B04", "B03", "B02"])
rgb

Great, our file size should be much smaller by removing all but 3 bands.

It can be somewhat tricky to display image composites nicely, especially if the data type is not already unsigned 8-bit integer (i.e. 0 - 255). In this case, our pixel values are `float64` (i.e. a much wider range than 8-bit). We can scale and combine bands using [xarray.spatial](https://xarray-spatial.org/index.html)'s [true_color()](https://xarray-spatial.org/reference/_autosummary/xrspatial.multispectral.true_color.html) method to scale and combine our values into an RGB composite.

In [ ]:
true_color = ms.true_color(*rgb[5])
plt.imshow(true_color)

Ultimately, we are going to create an animated GIF of our images.

In [ ]:
output_dir = "gif_output"

# create the directory if it does not exist
if not os.path.isdir(output_dir):
    os.makedirs(output_dir)

# for each image, save the true color image to the directory, as
# a zero-padded file name (e.g. img0001.png)
for i in range(rgb.shape[0]):
  print(f"Processing image: img{str(i).zfill(4)}.png")
  plt.imsave(f"{output_dir}/img{str(i).zfill(4)}.png", ms.true_color(*rgb[i]).compute().data)

[ffmpeg](https://ffmpeg.org/) is an open source system tool for processing audio and video data. We can install it on our Colab instance like:

In [ ]:
!apt install ffmpeg

Finally, we can create our animated GIF by stitching our image frames into a video. The following command:
- navigates into the `gif_output` directory
- runs the `ffmpeg` command:
  - format (-f): [image2](https://ffmpeg.org/ffmpeg-formats.html#image2-1) (a list of images)
  - input (-i): images matching the pattern `img%004d.png` (the string "img", followed by a 4-digit number, followed by the string ".png")
  - frame rate (-r): 10 frames per second
  - output path: `out.gif`
  - answer 'yes' to any prompts (-y): if the animation already exists, it will ask you to confirm overwriting, which is awkward in a notebook.

In the end, there should be an animation in the output directory.

In [ ]:
!cd gif_output && ffmpeg -f image2 -i img%004d.png -r 10 out.gif -y

# Extra Credit Ideas

- create a time lapse animation for a different location (e.g. the Old Fort landslide)
- try finding/loading imagery from a [different collection](https://planetarycomputer.microsoft.com/catalog) in the Microsoft Planetary Computer
- advanced: try interpolating daily images between two or more images
- create a time lapse animation showing different band combinations (e.g. false color)
- create a time lapse animation that uses a different function from xarray.spatial (e.g. [NDVI](https://xarray-spatial.org/user_guide/multispectral.html#NDVI))
- create a time lapse animation that applies a custom algorithm (e.g. [NDWI](https://en.wikipedia.org/wiki/Normalized_difference_water_index))
- check the [xarray gallery](https://docs.xarray.dev/en/stable/gallery.html) for inspiration
- [rioxarray](https://corteva.github.io/rioxarray/html/getting_started/getting_started.html) is another popular library for loading spatial images into xarray (rio = rasterio)
- try running the examples in [stackstac](https://stackstac.readthedocs.io/en/latest/)
- check out some of the STAC ecosystem:
  - if you use QGIS, try installing the [QGIS STAC API Browser](https://stac-utils.github.io/qgis-stac-plugin/) and loading images into QGIS directly
  - [STAC Index](https://stacindex.org/)
  - [stac-utils](https://github.com/stac-utils)
  - [staclint](https://staclint.com/) (Sparkgeo made this)
- check out the cloud-optimized geotiff tools in [cogeo.org](https://www.cogeo.org/)